Mount Google Drive env to import the modules and the dataset.



In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# Append the directory to our python path using sys

Now we are able to import stuff from that directory.

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ANFIS-imgSatellitari')
%cd /content/drive/MyDrive/ANFIS-imgSatellitari/

/content/drive/MyDrive/ANFIS-imgSatellitari


Import all the libraries and modules.



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from training import train_model
from testMod import test_model
import anfis
from membership import *
from utility import load_model, plot_import, split_dataset

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


cpu
Begin training.


# Test 1

In the following test I choose to configure the ANFIS training with:


*   Nr. 2 Fuzzy Sets
*   32 of Batch size
*   Nr. 200 of epochs

CLASSI NELLE METRICHE:
0 = BUILDING
1 = ROAD
2 = PAVEMENT
3 = VEGETATION
4 = BARE SOIL
5 = WATER



In [ ]:
dataset = 'datasets/reducedTopClass/reducedTC.csv'
model = None
n_terms = 2 #IMPOSTA NUMERO DI FUZZY SET
num_categories = 6 #IMPOSTA NUMERO DI CLASSI
batch_size = 32
epoch = 200
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)

model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(2) + 'topClass' + '.h5')

model = torch.load('models/G_model_geo_2topClass.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_2topClass.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf1 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf1 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf1 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf1 and x2 is mf1 and x3 is mf1
           THEN [[0.0], 

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
265360 of 353895 correct (accuracy=74.98%)
tensor([5, 3, 1,  ..., 1, 3, 2])
[[56634  3387  4407  3304   483    41]
 [ 6130 19420  4345  8471  2430   840]
 [ 5385  4974 47306  6204  1149   668]
 [ 6422  5069  1787 71608  4358  1735]
 [  812   334  5535  3886 28000    86]
 [  103   827    30  5264    69 42392]]
              precision    recall  f1-score   support

         0.0       0.75      0.83      0.79     68256
         1.0       0.57      0.47      0.51     41636
         2.0       0.75      0.72      0.73     65686
         3.0       0.73      0.79      0.75     90979
         4.0       0.77      0.72      0.75     38653
         5.0       0.93      0.87      0.90     48685

    accuracy                           0.75    353895
   macro avg       0.75      0.73      0.74    353895
weighted avg       0.75      0.75      0.75    353895



# Test 2

In the following test I choose to configure the ANFIS training with:


*   Nr. 3 Fuzzy Sets
*   32 of Batch size
*   Nr. 100 of epochs



In [ ]:
dataset = 'datasets/reducedTopClass/reducedTC.csv'
model = None
n_terms = 3 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 6 #IMPOSTA NUMERO DI CLASSI
epoch = 200
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(3) + 'topClass' + '.h5')

model = torch.load('models/G_model_geo_3topClass.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_3topClass.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf1
           THEN [[0.0], 

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
270742 of 353895 correct (accuracy=76.50%)
tensor([5, 3, 1,  ..., 1, 3, 2])
[[57255  2960  4354  3274   375    38]
 [ 5798 20077  4711  7794  2363   893]
 [ 5449  4879 48615  5011  1324   408]
 [ 6252  4860  2386 72245  4337   899]
 [  816   437  5232  2983 29094    91]
 [   88   595    57  4388   101 43456]]
              precision    recall  f1-score   support

         0.0       0.76      0.84      0.80     68256
         1.0       0.59      0.48      0.53     41636
         2.0       0.74      0.74      0.74     65686
         3.0       0.75      0.79      0.77     90979
         4.0       0.77      0.75      0.76     38653
         5.0       0.95      0.89      0.92     48685

    accuracy                           0.77    353895
   macro avg       0.76      0.75      0.75    353895
weighted avg       0.76      0.77      0.76    353895



# Test 3

In the following test I choose to configure the ANFIS training with:


*   Nr. 4 Fuzzy Sets
*   32 of Batch size
*   Nr. 100 of epochs



In [ ]:
dataset = 'datasets/reducedTopClass/reducedTC.csv'
model = None
n_terms = 4 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 6 #IMPOSTA NUMERO DI CLASSI
epoch = 200
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(4) + 'topClass' '.h5')

model = torch.load('models/G_model_geo_4topClass.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_4topClass.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf3
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf3
           THEN [[0.0], 

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
274390 of 353895 correct (accuracy=77.53%)
tensor([5, 3, 1,  ..., 1, 3, 2])
[[57420  3214  4201  2972   426    23]
 [ 5636 21771  3983  6771  2652   823]
 [ 5234  4364 49578  4543  1681   286]
 [ 6327  5069  2466 71491  4625  1001]
 [  825   528  4762  2431 30008    99]
 [   86   475   116  3766   120 44122]]
              precision    recall  f1-score   support

         0.0       0.76      0.84      0.80     68256
         1.0       0.61      0.52      0.57     41636
         2.0       0.76      0.75      0.76     65686
         3.0       0.78      0.79      0.78     90979
         4.0       0.76      0.78      0.77     38653
         5.0       0.95      0.91      0.93     48685

    accuracy                           0.78    353895
   macro avg       0.77      0.76      0.77    353895
weighted avg       0.77      0.78      0.77    353895

